<h1> Processing of the yeast data <span class="tocSkip"></span></h1>

# Introduction

Es necesario procesar los datos para reducir la memoria necesaria para el entrenamiento, especialmente en casos como los métodos estadísticos o basados en árboles que necesitan reducir la dimensionalidad para lograr un equilibrio eficiente entre tiempo, memoria y rendimiento durante el entrenamiento. Además, se generarán varios conjuntos de datos de test con distintos porcentajes de datos perdidos para evaluar el rendimiento de la imputación en estos distintos casos.


In [12]:
import numpy as np
import pandas as pd  
from matplotlib import pyplot as plt
import zipfile
import builtins  # Para asignar variables globales dinámicamente

### Descomprimir zip y guardar como df
Se sube al repositorio los datos comprimidos para obtimizar el almacenamiento. Por ende para tratar con los datos primero hay que descomprimirlos y guardarlos como DataFrame de pandas

In [13]:


def unzip(archivo_zip, variable_name):
    """
    Extrae y carga el contenido del único archivo dentro de un ZIP en una variable global.
    
    Parámetros:
    - archivo_zip: Ruta al archivo ZIP.
    - variable_name: Nombre de la variable global donde se guardará el DataFrame.
    """
    with zipfile.ZipFile(archivo_zip, 'r') as zip_ref:
        # Obtiene directamente el primer (y único) archivo
        file_name = zip_ref.namelist()[0]
        with zip_ref.open(file_name) as txt_file:
            # Leer el archivo como un DataFrame de pandas
            df = pd.read_csv(txt_file, sep='\t', index_col=0)
            
            # Asignar el DataFrame a una variable global con el nombre proporcionado
            builtins.__dict__[variable_name] = df

# paths de descompresión
zip_file_train= '../data/raw/yeast_genotype_train.txt.zip'
zip_file_test = '../data/raw/yeast_genotype_test.txt.zip'

# Llamar a la función para descomprimir y guardar los DataFrames en variables globales
unzip(zip_file_train, 'df_train') 
unzip(zip_file_test, 'df_test')   



In [14]:
#ver sets entrenamiento y prueba 
print("Entrenamiento:")
print(df_train.head(5))


print("\nPrueba:")
print(df_test.head(5))
df_train.shape, df_test.shape

Entrenamiento:
       33070_chrI_33070_A_T  33147_chrI_33147_G_T  33152_chrI_33152_T_C  \
SAMID                                                                     
01_01                     1                     1                     1   
01_02                     1                     1                     1   
01_03                     2                     2                     2   
01_04                     1                     1                     1   
01_06                     2                     2                     2   

       33200_chrI_33200_C_T  33293_chrI_33293_A_T  33328_chrI_33328_C_A  \
SAMID                                                                     
01_01                     1                     1                     1   
01_02                     1                     1                     1   
01_03                     2                     2                     2   
01_04                     1                     1                     1   
01_06    

((3513, 28220), (877, 28220))

In [15]:
#Visualizar datos
df_train.head(10)

,33070_chrI_33070_A_T,33147_chrI_33147_G_T,33152_chrI_33152_T_C,33200_chrI_33200_C_T,33293_chrI_33293_A_T,33328_chrI_33328_C_A,33348_chrI_33348_G_C,33403_chrI_33403_C_T,33502_chrI_33502_A_G,33548_chrI_33548_A_C,...,12048853_chrXVI_925593_G_C,12049199_chrXVI_925939_T_C,12049441_chrXVI_926181_C_T,12050613_chrXVI_927353_T_G,12051167_chrXVI_927907_A_C,12051240_chrXVI_927980_A_G,12051367_chrXVI_928107_C_T,12052782_chrXVI_929522_C_T,12052988_chrXVI_929728_A_G,12053130_chrXVI_929870_C_T
SAMID,,,,,,,,,,,,,,,,,,,,,
01_01,1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2
01_02,1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2
01_03,2,2,2,2,2,2,2,2,2,2,...,1,1,1,1,1,1,1,1,1,1
01_04,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
01_06,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
01_07,1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2
01_08,2,2,2,2,2,2,2,2,2,2,...,1,1,1,1,1,1,1,1,1,1
01_09,2,2,2,2,2,2,2,2,2,2,...,1,1,1,1,1,1,1,1,1,1
01_10,1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2


## Reducir columnas con varianza
Se seleccionan las 1000 columnas que presentan mayor varianza, considerándolas las que pueden enriquecer más el entrenamiento. El dataset original con más de 28000 dificulta en exceso la eficiencia del entrenamiento en modelos que requieren más cómputo por lo que la estrategia es reducir columnas


In [16]:
from sklearn.feature_selection import VarianceThreshold


selector = VarianceThreshold()

# Ajustar el selector y transformar los datos se pasa a np array)
X_train_reduced = selector.fit_transform(df_train)

# Obtener las varianzas calculadas
variances = selector.variances_

# Obtener los índices de las 1,000 características con mayor varianza
idx_sorted = np.argsort(variances)[-1000:]  

# Convertir de nuevo a DataFrame, seleccionando las 1,000 columnas obtenidas que tienen mayor varianza
df_train_reduced = pd.DataFrame(X_train_reduced[:, idx_sorted], columns=df_train.columns[idx_sorted])

#TEST
# Transformar el conjunto de test usando el mismo selector
X_test_reduced = selector.transform(df_test)

# Convertir de nuevo el array de test a DataFrame, seleccionando las mismas 1,000 columnas
df_test_reduced = pd.DataFrame(X_test_reduced[:, idx_sorted], columns=df_train.columns[idx_sorted])

print(f"Número de columnas seleccionadas: {df_train_reduced.shape[1]}")


Número de columnas seleccionadas: 1000


In [18]:
#revisar tras eliminar columnas
df_test_reduced, df_train_reduced

(     10335183_chrXV_303214_G_T  10341838_chrXV_309869_T_C  \
 0                            1                          1   
 1                            1                          1   
 2                            2                          2   
 3                            2                          2   
 4                            1                          1   
 ..                         ...                        ...   
 872                          2                          2   
 873                          1                          1   
 874                          2                          1   
 875                          2                          2   
 876                          1                          1   
 
      10341923_chrXV_309954_G_A  10342190_chrXV_310221_T_C  \
 0                            1                          1   
 1                            1                          1   
 2                            2                          2   
 3    

# Method

In [19]:
# Convertir df_test a categoría si es necesario
#df_test = df_test_reduced.astype('category')


def introduce_missingness(data, missing_perc=0.1):
    """
    Introduce valores faltantes (NaN) en un porcentaje determinado de celdas en el DataFrame.
    """
    data_missing = data.copy()

    # Calcular el número de valores a modificar
    n_samples, n_features = data_missing.shape
    n_missing = int(np.floor(n_samples * n_features * missing_perc))

    # Seleccionar aleatoriamente las posiciones donde se introducirán los valores faltantes (NaN)
    missing_indices = np.random.choice(n_samples * n_features, n_missing, replace=False)

    # Introducir los valores faltantes (NaN) en las posiciones seleccionadas
    for index in missing_indices:
        i = index // n_features
        j = index % n_features
        data_missing.iloc[i, j] = np.nan  # Introducir NaN como valor faltante

    return data_missing

# Aplicar la función al conjunto de test, con distinto porcentaje de pérdida
df_test_missing_10 = introduce_missingness(df_test_reduced, missing_perc=0.1)
df_test_missing_20 = introduce_missingness(df_test_reduced, missing_perc=0.2)
df_test_missing_30= introduce_missingness(df_test_reduced, missing_perc=0.3)
df_test_missing_40 = introduce_missingness(df_test_reduced, missing_perc=0.4)

In [20]:
#revisar tras introducir NaN 
df_test_missing_10

,10335183_chrXV_303214_G_T,10341838_chrXV_309869_T_C,10341923_chrXV_309954_G_A,10342190_chrXV_310221_T_C,10342543_chrXV_310574_A_G,10344120_chrXV_312151_C_T,10346156_chrXV_314187_C_G,10349298_chrXV_317329_C_T,3141305_chrV_249350_C_T,10383039_chrXV_351070_C_T,...,10640336_chrXV_608367_C_T,1428699_chrIV_68677_T_C,1428933_chrIV_68911_A_C,6970183_chrXI_391971_C_T,6970060_chrXI_391848_C_T,6969970_chrXI_391758_T_A,3698349_chrVI_229520_A_G,6969924_chrXI_391712_C_T,5535875_chrIX_143302_C_T,7948843_chrXII_703815_C_T
0,1.0,NaN,1.0,1.0,1.0,1.0,NaN,1.0,1.0,2.0,...,2.0,1.0,1.0,1.0,1.0,NaN,2.0,1.0,1.0,1.0
1,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,2.0,1.0,...,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0
2,2.0,2.0,NaN,2.0,NaN,NaN,2.0,2.0,1.0,2.0,...,2.0,NaN,1.0,1.0,1.0,NaN,1.0,1.0,2.0,1.0
3,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,...,NaN,2.0,2.0,NaN,2.0,2.0,2.0,2.0,NaN,2.0
4,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,1.0,1.0,2.0,2.0,2.0,2.0,NaN,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,...,1.0,1.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0
873,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,2.0,...,NaN,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0
874,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,...,NaN,1.0,1.0,1.0,1.0,1.0,2.0,1.0,NaN,1.0
875,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,1.0,1.0,1.0,2.0,2.0,2.0,NaN,2.0,2.0,2.0


In [21]:
#Pasar a str para evitar problemas de formatos en exportación
df_train_reduced.columns= df_train_reduced.columns.map(str)
df_test_reduced.columns= df_test_reduced.columns.map(str)
df_test_missing_10.columns= df_test_missing_10.columns.map(str)
df_test_missing_20.columns= df_test_missing_20.columns.map(str)
df_test_missing_30.columns= df_test_missing_30.columns.map(str)
df_test_missing_40.columns= df_test_missing_40.columns.map(str)

### Guardar archivos resultantes
Guardamos en un formato de datos muy eficiente y optimizado (.parquet) los datos para cargarlos de cara a entrenar y testear los distintos modelos, usando exactamente los mismos datos y haciendo el proceso completamente reproducible

In [22]:

df_train_reduced.to_parquet('../data/processed/df_train_reduced.parquet')
df_test_reduced.to_parquet('../data/processed/df_test_reduced.parquet')
df_test_missing_10.to_parquet('../data/processed/df_test_reduced_missing_10.parquet')
df_test_missing_20.to_parquet('../data/processed/df_test_reduced_missing_20.parquet')
df_test_missing_30.to_parquet('../data/processed/df_test_reduced_missing_30.parquet')
df_test_missing_40.to_parquet('../data/processed/df_test_reduced_missing_40.parquet')
